<center><a href="https://www.nvidia.com/dli"> <img src="images/DLI_Header.png" alt="Header" style="width: 400px;"/> </a></center>




# 5a. 預訓練模型(Pre-Trained Models)


雖然解決深度學習挑戰通常需要一個大型且標註良好的資料集，但有許多可以直接使用的免費預訓練模型。當您決定開始自己的深度學習專案時，先尋找可以幫助您達成目標的現有模型是個很好的想法。探索可用模型的絕佳地點是 [NGC](https://ngc.nvidia.com/catalog/models)。此外，您還可以通過 Google 搜尋找到許多託管(host)在 GitHub 上的模型。

## 5a.1 目標(Objectives)

-   使用 TorchVision 載入一個訓練良好的預訓練模型(pretrained model)
-   預處理我們自己的圖像以配合預訓練模型使用
-   使用預訓練模型對您自己的圖像進行準確的推論(inference)

In [ ]:
import torch
import torchvision.transforms.v2 as transforms
import torchvision.io as tv_io

import json

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
torch.cuda.is_available()

## 5a.2 自動狗門


在本節中，我們將創建一個只允許狗（而非其他動物）進出的狗門。我們可以讓我們的貓留在室內，而其他動物則留在應該待的室外。使用到目前為止所介紹的技術，我們需要一個非常大的資料集，其中包含許多狗的圖片以及其他動物。幸運的是，有一個現成的模型已經在包含大量動物的海量資料集上進行了訓練。

[ImageNet 挑戰賽](https://en.wikipedia.org/wiki/ImageNet#History_of_the_ImageNet_challenge)已經產生了許多可用於圖像分類的最先進模型。這些模型在數百萬張圖像上進行訓練，可以準確地將圖像分類為 1000 個不同類別。其中許多類別是動物，包括各種品種的狗和貓。這對我們的狗門來說是一個完美的模型。

## 5a.3 載入模型(Loading the Model)


我們將首先下載模型。已訓練的 ImageNet 模型可以直接從 TorchVision 函式庫中下載。我們可以在[這裡](https://pytorch.org/vision/stable/models.html)查看可用的模型及其詳細資訊。這些模型中的任何一個都適用於我們的練習。我們將選擇一個常用的模型，稱為 [VGG16](https://pytorch.org/vision/stable/models/vgg.html)，並使用[預設權重(default weights)](https://pytorch.org/vision/stable/models/generated/torchvision.models.vgg19.html#torchvision.models.VGG19_Weights)。

In [ ]:
from torchvision.models import vgg16
from torchvision.models import VGG16_Weights

# load the VGG16 network *pre-trained* on the ImageNet dataset
weights = VGG16_Weights.DEFAULT
model = vgg16(weights=weights)

現在模型已經載入，讓我們來看看它。它看起來很像我們在手語練習中使用的卷積模型(convolutional model)。

In [ ]:
model.to(device)


### 5a.3.1 輸入維度(Input dimensions)

與我們之前的練習一樣，我們的圖像應該符合模型預期的輸入維度。由於 PyTorch 模型是動態構建的，模型本身不知道其輸入形狀應該是什麼。幸運的是，預訓練的`權重(weights)`帶有自己的轉換器(transforms)。

In [ ]:
pre_trans = weights.transforms()
pre_trans

這相當於以下內容：

```Python
IMG_WIDTH, IMG_HEIGHT = (224, 224)

pre_trans = transforms.Compose([
    transforms.ToDtype(torch.float32, scale=True), # Converts [0, 255] to [0, 1]
    transforms.Resize((IMG_WIDTH, IMG_HEIGHT)),
    transforms.Normalize(
        mean=[0.485, 0.456, 0.406],
        std=[0.229, 0.224, 0.225],
    ),
    transforms.CenterCrop(224)
])
```

這裡有一個新的轉換器(Transform)是 [Normalize](https://pytorch.org/vision/main/generated/torchvision.transforms.Normalize.html)，它將根據提供的[平均值(mean)](https://en.wikipedia.org/wiki/Mean)和[標準差(standard deviation)](https://en.wikipedia.org/wiki/Standard_deviation)重新著色圖像。另一個新的轉換器(Transform)是 [CenterCrop](https://pytorch.org/vision/stable/generated/torchvision.transforms.v2.CenterCrop.html#torchvision.transforms.v2.CenterCrop)，它會移除圖像的邊緣。我們可以將這些轉換應用於任何我們想要與 VGG16 模型一起使用的圖像張量(tensor)。

### 5a.3.2 輸出維度(Output dimensions)


我們還可以看到模型將回傳一個形狀(shape)為 1000 的預測結果。請記住，在我們的第一個練習中，模型的輸出形狀(shape)是 10，對應於 10 個不同的數字。在我們的第二個練習中，形狀(shape)為 24，對應於可以在靜態圖像中捕捉的手語字母表的 24 個字母。在這裡，我們有 1000 個可能的類別，圖像將被分類到其中之一。雖然完整的 ImageNet 資料集有超過 20,000 個類別，但競賽和由此產生的預訓練模型只使用了這些類別中的 1000 個子集。我們可以在[這裡查看所有這些可能的類別](https://gist.github.com/yrevar/942d3a0ac09ec9e5eb3a)。

許多類別是動物，包括許多種類的狗和貓。狗是類別 151 到 268。貓是類別 281 到 285。我們將能夠使用這些類別來告訴我們的狗門門口是什麼類型的動物，以及我們是否應該讓它們進入。

## 5a.4 載入圖像(Loading an Image)

我們將首先載入一張圖像並顯示它，就像我們在之前的練習中所做的那樣。

In [ ]:
import matplotlib.pyplot as plt
import matplotlib.image as mpimg

def show_image(image_path):
    image = mpimg.imread(image_path)
    plt.imshow(image)

In [ ]:
show_image("data/doggy_door_images/happy_dog.jpg")

### 5a.4.1 預處理圖像(Preprocessing the Image)


接下來，我們將預處理圖像，使其準備好送入模型。這就像我們在上一個練習中對手語圖像進行預測時所做的一樣。請記住，在這種情況下，圖像的最終形狀應該是 (1, 3, 224, 224)。我們將使用 `weights` 提供的轉換器(Transform)來完成這個操作。

In [ ]:
def load_and_process_image(file_path):
    # Print image's original shape, for reference
    print('Original image shape: ', mpimg.imread(file_path).shape)
    
    image = tv_io.read_image(file_path).to(device)
    image = pre_trans(image)  # weights.transforms()
    image = image.unsqueeze(0)  # Turn into a batch
    return image

讓我們在我們的快樂狗狗照片上練習，看看是否有效：

In [ ]:
processed_image = load_and_process_image("data/doggy_door_images/happy_dog.jpg")
print("Processed image shape: ", processed_image.shape)


形狀是正確的，但它看起來如何？讓我們印出一張圖像來驗證：

In [ ]:
import torchvision.transforms.functional as F

plot_image = F.to_pil_image(torch.squeeze(processed_image))
plt.imshow(plot_image, cmap='gray')

圖片看起來有一種迷幻效果，但如果我們瞇著眼睛，仍然可以看到我們快樂的狗。奇怪的著色是由於 `Normalize` 轉換，而照片看起來有點放大是因為 `CenterCrop`。


## 5a.5 進行預測(Make a Prediction)


現在我們的圖像格式正確，我們可以將其傳入模型並獲得預測結果。我們預期輸出是一個包含 1000 個元素的陣列，這將很難閱讀。我們有一個以 [json](https://www.json.org/json-en.html) 格式的所有類別列表，這類似於 python 列表(list)和字典(dictionary)的組合。事實上，讓我們將它載入到列表和字典的組合中。

In [ ]:
vgg_classes = json.load(open("data/imagenet_class_index.json"))



這個檔案使用數字(資料類別是string)對應每個類別：

In [ ]:
vgg_classes["0"]


讓我們創建一個函式，使 VGG 模型的預測結果易於人類閱讀。這類似於上一課中的 `predict_letter` 函式。這次，我們將使用 [torch.topk](https://pytorch.org/docs/stable/generated/torch.topk.html) 函式來給我們前 `3` 名的預測結果。

In [ ]:
def readable_prediction(image_path):
    # Show image
    show_image(image_path)
    # Load and pre-process image
    image = load_and_process_image(image_path)
    # Make predictions
    output = model(image)[0]  # Unbatch
    predictions = torch.topk(output, 3)
    indices = predictions.indices.tolist()
    # Print predictions in readable form
    out_str = "Top results: "
    pred_classes = [vgg_classes[str(idx)][1] for idx in indices]
    out_str += ", ".join(pred_classes)
    print(out_str)

    return predictions


在幾種動物上試試看結果！您也可以隨意上傳自己的圖像並對其進行分類，看看效果如何。

In [ ]:
readable_prediction("data/doggy_door_images/happy_dog.jpg")

In [ ]:
readable_prediction("data/doggy_door_images/brown_bear.jpg")

In [ ]:
readable_prediction("data/doggy_door_images/sleepy_cat.jpg")

## 5a.6 只允許狗(Only Dogs)

現在我們正在使用模型進行預測，我們可以使用我們的分類結果來只讓狗進出，並讓貓留在室內。狗是類別 151 到 268，貓是類別 281 到 285。

#### 練習(Exercise)



我們在上一課中使用了這個 [argmax](https://numpy.org/doc/stable/reference/generated/numpy.argmax.html) 函式。您還記得我們從哪個維度提取索引嗎？

**提示**：第一個維度是批次維度(batch dimension)。

In [ ]:
def doggy_door(image_path):
    show_image(image_path)
    image = load_and_process_image(image_path)
    idx = model(image).argmax(dim=FIXME).item()
    print("Predicted index:", idx)
    if 151 <= idx <= 268:
        print("Doggy come on in!")
    elif 281 <= idx <= 285:
        print("Kitty stay inside!")
    else:
        print("You're not a dog! Stay outside!")

### 解答(Solution)

點擊下方的 `...` 查看解答。

In [ ]:
# SOLUTION
import numpy as np

def doggy_door(image_path):
    show_image(image_path)
    image = load_and_process_image(image_path)
    idx = model(image).argmax(dim=1).item()
    print("Predicted index:", idx)
    if 151 <= idx <= 268:
        print("Doggy come on in!")
    elif 281 <= idx <= 285:
        print("Kitty stay inside!")
    else:
        print("You're not a dog! Stay outside!")

In [ ]:
doggy_door("data/doggy_door_images/brown_bear.jpg")

In [ ]:
doggy_door("data/doggy_door_images/happy_dog.jpg")

In [ ]:
doggy_door("data/doggy_door_images/sleepy_cat.jpg")

## 總結(Summary)

做得好！使用強大的預訓練模型，我們只用了幾行程式碼就創建了一個可以運作的狗門偵測模型。我們希望您對不需要大量前期工作就能利用深度學習感到興奮。最好的部分是，隨著深度學習社群的發展，將有更多模型可供您在自己的專案中使用。

### 清空記憶體(Clear the Memory)

在繼續之前，請執行以下程式碼區塊(cell)以清空 GPU 記憶體。

In [ ]:
import IPython
app = IPython.Application.instance()
app.kernel.do_shutdown(True)

## 下一步(Next)


使用預訓練模型非常強大，但有時它們可能不完全適合您的資料。在下一節中，您將了解另一種強大的技術，*遷移學習(transfer learning)*，它允許您調整預訓練模型以對您的資料做出良好的預測。

繼續下一節：[*預訓練模型(Pretrained Models)*](./05b_presidential_doggy_door.ipynb)。

<center><a href="https://www.nvidia.com/dli"> <img src="images/DLI_Header.png" alt="Header" style="width: 400px;"/> </a></center>


